In [1]:
import tensorflow as tf
import scipy.misc
import numpy as np
import scipy.misc
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

import cv2
from subprocess import call



img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

Using TensorFlow backend.


In [2]:
print(cv2.version)

AttributeError: 'module' object has no attribute 'version'

In [3]:

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
#read data.txt
with open("/home/dhanshri/Documents/DeepLearning/Midterm Project/Self Driving/coding stage 1/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("/home/dhanshri/Documents/DeepLearning/Midterm Project/Self Driving/coding stage 1/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)

#shuffle list of images
c = list(zip(xs, ys))
random.shuffle(c)
xs, ys = zip(*c)

train_xs = xs[:int(len(xs) * 0.08)]
train_ys = ys[:int(len(xs) * 0.08)]

val_xs = xs[-int(len(xs) * 0.02):]
val_ys = ys[-int(len(xs) * 0.02):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [60, 80]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [60, 80]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [4]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

#compile loaded model
loaded_model.compile(optimizer=Adam(lr=1e-4), loss = 'mse')


Loaded model from disk


In [7]:


i = 0
while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [60, 80]) / 255.0
    degrees = loaded_model.predict(XTest) * 180.0 / scipy.pi
    call("clear")
    print("Predicted steering angle: " + str(degrees) + " degrees")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


error: /io/opencv/modules/highgui/src/window.cpp:654: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvWaitKey


In [5]:
# predict on single file
xTest_batch,yTest_batch=LoadValBatch(1)
XTest=np.array(xTest_batch)
YTest=np.array(yTest_batch)

In [6]:
XTest.shape


(1, 60, 80, 3)

In [6]:
degrees = loaded_model.predict(XTest) * 180.0 / scipy.pi

In [8]:
i=1
full_image = scipy.misc.imread("/home/dhanshri/Documents/DeepLearning/Midterm Project/Self Driving/coding stage 1/Autopilot-TensorFlow-master/driving_dataset/" + str(i) + ".jpg", mode="RGB")

cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))

error: /io/opencv/modules/highgui/src/window.cpp:583: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvShowImage


In [8]:
full_image = scipy.misc.imread("/home/dhanshri/Documents/DeepLearning/Midterm Project/Self Driving/coding stage 1/Autopilot-TensorFlow-master/driving_dataset/" + str(i) + ".jpg", mode="RGB")
image = scipy.misc.imresize(full_image[-150:], [60, 80]) / 255.0
degrees = loaded_model.predict(XTest) * 180.0 / scipy.pi
call("clear")
print("Predicted steering angle: " + str(degrees) + " degrees")
cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
  #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
dst = cv2.warpAffine(img,M,(cols,rows))
cv2.imshow("steering wheel", dst)

Predicted steering angle: [[-1.85597634]] degrees


error: /io/opencv/modules/highgui/src/window.cpp:583: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvShowImage


In [33]:
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
dst = cv2.warpAffine(img,M,(cols,rows))
cv2.imshow("steering wheel", dst)

error: /io/opencv/modules/imgproc/src/color.cpp:9710: error: (-215) depth == CV_8U || depth == CV_16U || depth == CV_32F in function cvtColor


In [21]:
full_image = scipy.misc.imread("/home/dhanshri/Documents/DeepLearning/Midterm Project/Self Driving/coding stage 1/Autopilot-TensorFlow-master/driving_dataset/" + str(1) + ".jpg", mode="RGB")
image = scipy.misc.imresize(full_image[-150:], [60, 80]) / 255.0
degrees = loaded_model.predict(image) * 180.0 / scipy.pi

Exception: Error when checking : expected convolution2d_input_4 to have 4 dimensions, but got array with shape (60, 80, 3)